In [1]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd

In [16]:
path="/content/ANERCorp.xlsx"

In [17]:
xlsx = pd.ExcelFile(path)

In [18]:
df=pd.read_excel(xlsx, header=None)

In [21]:
df

,0,1,2
0,فرانكفورت,B-LOC,NaN
1,(د,O,NaN
2,ب,O,NaN
3,أ),O,NaN
4,أعلن,O,NaN
...,...,...,...
150281,الشخصية,O,NaN
150282,للوليد,B-PERS,NaN
150283,بن,I-PERS,NaN
150284,طلال,I-PERS,NaN


In [22]:
df = df.drop(2, 1)

In [23]:
df = df.rename(columns={0: 'text', 1: 'label'})

In [24]:
df

,text,label
0,فرانكفورت,B-LOC
1,(د,O
2,ب,O
3,أ),O
4,أعلن,O
...,...,...
150281,الشخصية,O
150282,للوليد,B-PERS
150283,بن,I-PERS
150284,طلال,I-PERS


In [25]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [26]:
train_arr=train['text'].astype(str)
train_lbl=train['label'].astype(str)
test_arr=test['text'].astype(str)
test_lbl=test['label'].astype(str)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer

In [28]:
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)

In [29]:
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)

In [30]:
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)

In [31]:
del df
del test_arr
del train_arr

In [32]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

In [33]:
train_tfmat

<120228x28206 sparse matrix of type '<class 'numpy.float64'>'
	with 107945 stored elements in Compressed Sparse Row format>

In [34]:
from sklearn.svm import LinearSVC

In [35]:
lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)

LinearSVC()

In [36]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [37]:
test=['ألمانيا']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'B-LOC'

In [38]:
from sklearn.metrics import  accuracy_score
from sklearn import metrics

In [39]:
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

accuracy: 0.9382859804378202


In [40]:
phrase="شاهد أحمد مباراة فرنسا"
arr=phrase.split()

In [41]:
arr

['شاهد', 'أحمد', 'مباراة', 'فرنسا']

In [42]:
import sys
y=[]
token=[]
for x in arr:
    x=[x]
    test_str = vectorizer.transform(x)
    test_tfstr = tfidf.transform(test_str)
    test_tfstr.shape
    token.append(x)
    y.append(lsvm.predict(test_tfstr.toarray())[0])

In [43]:
df=pd.DataFrame(list(zip(token,y)),columns=['token','entity_type'])

In [44]:
df

,token,entity_type
0,[شاهد],O
1,[أحمد],B-PERS
2,[مباراة],O
3,[فرنسا],B-LOC
